

# Caso de Estudio "Yapo"



### Implementación de una base de datos que almacena anuncios Yapo de la región de Los Ríos.

---

__PARTE PRÁCTICA__

### Pregunta 3:

* Implementar un script Python para crear la estructura de la base de datos ___Yapo-LosRios___

In [1]:
import mysql.connector
db_connection = mysql.connector.connect(user="root",host="localhost",password="#Sombrita2019")
cursor = db_connection.cursor()

In [12]:
cursor.execute("DROP DATABASE Yapo;")

In [13]:
cursor.execute("CREATE DATABASE Yapo;")
cursor.execute("USE Yapo")

#tabla vendedor
cursor.execute("CREATE TABLE vendedor ("+
               "id_vendedor VARCHAR(130) PRIMARY KEY, nombre VARCHAR(100), ciudad VARCHAR(30), codigo_region VARCHAR(7), "+
               "nombre_region VARCHAR(30), "+
               "fecha_inscripcion DATE)")

#tabla anuncio
cursor.execute("CREATE TABLE anuncio (url VARCHAR(300) PRIMARY KEY, "+
               "titulo VARCHAR(200), descripcion MEDIUMTEXT, precio INT, moneda VARCHAR(2), fecha_publicacion DATE, categoria VARCHAR(50), "
               +"id_vendedor VARCHAR(130), FOREIGN KEY (id_vendedor) REFERENCES vendedor(id_vendedor))")

---

### Pregunta 4:


Implementar un script Python para llenar la base de datos “Yapo-LosRios” escrapeando algunos datos de Yapo.cl

In [4]:
#!pip install beautifulsoup4
#!pip install selenium

In [14]:
from urllib.request import urlopen as uReq
import urllib.request

from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
#Selenium -> automatizar la navegación en el navegador Chrome
#BeautifulSoup -> Navegar en un archivo HTML

In [15]:
browser = webdriver.Firefox()
myUrl = 'https://www.yapo.cl/chile/todos_los_avisos?ca=12_s&l=0'
browser.get(myUrl)

In [16]:
pageSoup = soup(browser.page_source, 'html.parser')
pages = pageSoup.find('span',  {'class', 'nohistory FloatRight'}).a['href']

In [17]:
pages

'https://www.yapo.cl/chile?ca=12_s&o=28193'

In [18]:
index = pages.rfind('=')
print(index)
lastPage = int(pages[index+1:])
print(lastPage)
root_pages = pages[:index+1]
print(root_pages)

35
28193
https://www.yapo.cl/chile?ca=12_s&o=


* Conversión de fechas: 

In [19]:
import datetime

def convert_date(yapo_date):
    month=yapo_date.split(" ")[0]
    year=yapo_date.split(" ")[1]
    
    convert = {'Enero' : 1,
            'Febrero' : 2,
            'Marzo' : 3,
            'Abril' : 4,
            'Mayo' : 5,
            'Junio' : 6,
            'Julio' : 7,
            'Agosto' : 8,
            'Septiembre' : 9, 
            'Octubre' : 10,
            'Noviembre' : 11,
            'Diciembre' : 12
           }
    
    
    new_date=datetime.date(int(year), convert[month], 1) 

    return new_date

* Se empieza a iterar en cada pagina(scraping) para llenar la base de datos.

In [20]:
for i in range(lastPage):
    
    #recuperarmos la URL de la pagina corriente
    url = root_pages + str(i+1)
    
    #con Selenium, vamos en navegar en esta página
    browser.get(url)
    
    #empezamos el scrapping de la pagina corriente
    pageSoup = soup(browser.page_source, 'html.parser')
    
    #recuperamos todos los tags HTML que corresponden a la lista de anuncios en esta pagina
    links = pageSoup.findAll('td', {'class' : 'thumbs_subject'})
    
    #empezamos a iterar sobre cada anuncio
    for link in links:
        
        #todos los datos que necesitamos encontrar
        url, titulo, descripcion, precio, fecha_publicacion, categoria="","","",None,"", ""
        nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion="","","","",""
        
        #Navegamos hacia la pagina del anuncio
        url=link.find('a',{'class':'title'})['href']
        print(url)
        browser.get(link.find('a',{'class':'title'})['href'])

        #RECUPERAMOS LA FECHA DE PUBLICACION DEL ANUNCIO          
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('time')):
            datetime_raw = pageSoup.find('time').attrs['datetime']
            date_publication_raw=datetime_raw.split("T")[0]
            date_publication_raw=  date_publication_raw.split("-")
            
            date_publication=datetime.date(int(date_publication_raw[0]), int(date_publication_raw[1]),
                                           int(date_publication_raw[2]))
            print("datetime:")
            print(date_publication)
        
        #RECUPERAMOS EL TITULO DEL ANUNCIO           
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('h1', {"id" : "da_subject"})):
            titulo = pageSoup.find('h1', {"id" : "da_subject"}).text.strip()
            print(titulo)
            
        #RECUPERAMOS LA DESCRIPCION DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "description"})):
            try:
                descripcion = pageSoup.find('div', {"class" : "description"}).text.split(' ', 1)[1].strip().replace(u'\n', u' ')
            except:
                continue
                
        #RECUPERAMOS EL PRECIO DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "price text-right"})):
            precio_raw = pageSoup.find('div', {"class" : "price text-right"}).text.strip().replace(u'\n', u' ').replace(u'\t', u'')
            precio_raw = precio_raw.split(" ")
            moneda = precio_raw[0]
            precio = precio_raw[1].split(",")[0].replace(u'.', u'')
            print("precio:"+precio)
            
        #RECUPERAMOS LA CATEGORIA DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "breadcrumbs"})):
            categoria = pageSoup.find('div', {"class" : "breadcrumbs"}).find('a', {"id" : "breadcrumb_category"}).find('strong').text.strip().replace(u'\n', u' ')
            print(categoria)
            
        
        #RECUPERAMOS EL NOMBRE DEL VENDEDOR, SU FECHA DE INSCRIPCION Y SU LOCALIDAD
        if(pageSoup.find('aside', {"class" : "sidebar-right"})):
            aside = pageSoup.find('aside', {"class" : "sidebar-right"})
    
            #print(aside.find('seller-info'))
            
            #NOMBRE
            if(aside.find('seller-info')!=None):
                nombre=aside.find('seller-info').attrs['username']
            
                #FECHA DE INSCRIPCION
                fecha_inscripcion_raw=aside.find('seller-info').attrs['seniority']
                try:
                    fecha_inscripcion_raw=fecha_inscripcion_raw[len("En Yapo desde "):]
                    fecha_inscripcion=convert_date(fecha_inscripcion_raw)
                    print(fecha_inscripcion)
                except:
                    fecha_inscripcion=None
                    continue
                
                #LOCALIDAD
                localidad_raw=aside.find('seller-info').attrs['region']
            
                #print(localidad_raw)
            
                region_raw=localidad_raw.split(",")[0]
                ciudad_raw=localidad_raw.split(",")[1]
            
                codigo_region=region_raw.split(" ")[0]
                nombre_region=region_raw[len(codigo_region)+1:]
                ciudad=ciudad_raw[1:]
                
        # LLENAMOS LA BASE DE DATOS: TABLA VENDEDOR
        
        try:
            sql = "INSERT INTO vendedor (id_vendedor,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion) VALUES (%s,%s, %s, %s,%s,%s)"
            val = (nombre+"_"+codigo_region+"_"+ciudad,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion)
            cursor.execute(sql, val)
        except Exception as e1:
            print(e1)
            #continue
        
        # LLENAMOS LA BASE DE DATOS: TABLA ANUNCIO
        try:
            sql = "INSERT INTO anuncio (url, titulo, descripcion, precio, moneda, fecha_publicacion, categoria, id_vendedor) VALUES (%s, %s, %s,%s, %s, %s, %s, %s)"
            val = (url, titulo, descripcion, int(precio), moneda, date_publication, categoria, nombre+"_"+codigo_region+"_"+ciudad)
            cursor.execute(sql, val)
        except Exception as e2:
            print(e2)
            #continue
        
        cursor.execute("COMMIT")

https://www.yapo.cl/region_metropolitana/moda-vestuario/polera__74023775.htm?ca=15_s&first=1&oa=74023775&xsp=0
datetime:
2020-08-28
Polera
precio:2000
Moda y vestuario
2014-05-01
https://www.yapo.cl/region_metropolitana/bolsos-bisuteria-accesorios/elegante_cartera_marca_river_island_74023774.htm?ca=15_s&oa=74023774&xsp=1
datetime:
2020-08-28
Elegante cartera marca River Island
precio:10000
Bolsos, bisutería y accesorios
2014-05-01
https://www.yapo.cl/region_metropolitana/calzado/botin_cafe_74023773.htm?ca=15_s&oa=74023773&xsp=2
datetime:
2020-08-28
Botin cafe
precio:10000
Calzado
2017-10-01
https://www.yapo.cl/region_metropolitana/comprar/inversion_1dormtirio__metro_parque_almagro_74023772.htm?ca=15_s&oa=74023772&xsp=3
datetime:
2020-08-28
INVERSIÓN 1dormtirio, Metro Parque Almagro
precio:2200
Vendo - Departamento
2015-03-01
https://www.yapo.cl/antofagasta/autos/mitsubishi_lancer_2008_74023771.htm?ca=3_s&oa=74023771&xsp=4
datetime:
2020-08-28
Mitsubishi lancer 2008
precio:3600000
Autos

https://www.yapo.cl/valparaiso/comprar/casa_agua_santa_vina_del_mar_70408358.htm?ca=6_s&oa=70408358&xsp=37
datetime:
2020-08-28
CASA Agua Santa Viña del Mar
precio:130000000
Vendo - Casa
2020-08-01
https://www.yapo.cl/region_metropolitana/comprar/parcela_atajo_del_horno_curacavi_70429000.htm?ca=15_s&oa=70429000&xsp=38
datetime:
2020-08-28
PARCELA Atajo del Horno Curacaví
precio:11580
Vendo - Terreno
2020-08-01
https://www.yapo.cl/region_metropolitana/comprar/departamento_metro_san_alberto_hurtado__con__8230__74023741.htm?ca=15_s&oa=74023741&xsp=39
datetime:
2020-08-28
Departamento METRO SAN ALBERTO HURTADO, CON…
precio:63000000
Vendo - Departamento
2016-12-01
https://www.yapo.cl/region_metropolitana/comprar/casa_arrayan_lo_barnechea_70400764.htm?ca=15_s&oa=70400764&xsp=40
datetime:
2020-08-28
CASA ARRAYAN Lo Barnechea
precio:17000
Vendo - Casa
2020-08-01
https://www.yapo.cl/region_metropolitana/arrendar/remodelado_jorge_washington_74023740.htm?ca=15_s&oa=74023740&xsp=41
datetime:
2020-

https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/kia_avella_ano_2000_alternador_74023712.htm?ca=15_s&oa=74023712&xsp=24
datetime:
2020-08-28
Kia Avella año 2000 alternador
precio:29900
Accesorios y piezas para vehículos
2020-02-01
https://www.yapo.cl/region_metropolitana/television_camaras/audifonos_bluetooth_v5_0_i11_inalambrico_74023711.htm?ca=15_s&oa=74023711&xsp=25
datetime:
2020-08-28
Audifonos Bluetooth V5.0 I11 Inalámbrico
precio:14990
Audio, TV, video y fotografía
2020-08-01
https://www.yapo.cl/biobio/camiones_furgones/camion_foodtruck_dfsk_cargo_box_c21_74023710.htm?ca=9_s&oa=74023710&xsp=26
datetime:
2020-08-28
Camion Foodtruck,dfsk,cargo Box C21
precio:5000000
Buses, camiones y furgones
2016-02-01
https://www.yapo.cl/valparaiso/electrodomesticos/estufa_mademsa_mfhk540_74023709.htm?ca=6_s&oa=74023709&xsp=27
datetime:
2020-08-28
estufa mademsa MFHK540
precio:150000
Electrodomésticos
2016-06-01
https://www.yapo.cl/tarapaca/arrendar/habitacion__74023708.htm?ca=2_s&o

datetime:
2020-08-28
Barra de techo lado chofer nissan qasqai con tapa
precio:50000
Accesorios y piezas para vehículos
2014-02-01
https://www.yapo.cl/region_metropolitana/muebles/kardex_metalico_de_4_cajones_74023698.htm?ca=15_s&oa=74023698&xsp=8
datetime:
2020-08-28
Kardex metalico de 4 cajones
precio:50000
Muebles
2015-02-01
https://www.yapo.cl/region_metropolitana/arrendar/casa_el_remanso_las_condes_73396941.htm?ca=15_s&oa=73396941&xsp=9
datetime:
2020-08-28
CASA El Remanso Las Condes
precio:2400000
Arriendo - Casa
2020-08-01
https://www.yapo.cl/region_metropolitana/arrendar/parcela_camino_san_vicente___camino_la_lech__8230__73052500.htm?ca=15_s&oa=73052500&xsp=10
datetime:
2020-08-28
PARCELA Camino San Vicente / Camino La Lech…
precio:1050000
Arriendo - Terreno
2019-07-01
https://www.yapo.cl/region_metropolitana/ofertas_de_empleo/secretaria_auxiliar_74023697.htm?ca=15_s&oa=74023697&xsp=11
datetime:
2020-08-28
Secretaria auxiliar
Ofertas de empleo
2018-12-01
int() argument must be a

datetime:
2020-08-28
Parlante Bazooka philco bluetooth
precio:12990
Audio, TV, video y fotografía
2020-08-01
1062 (23000): Duplicate entry 'Leonardo rincon_Región_Santiago' for key 'vendedor.PRIMARY'
https://www.yapo.cl/valparaiso/autos/chevrolet_spark_2010_d_h_800_cc_74023676.htm?ca=6_s&oa=74023676&xsp=40
datetime:
2020-08-28
CHEVROLET SPARK 2010 D/H 800 cc
precio:2750000
Autos, camionetas y 4x4
2018-09-01
https://www.yapo.cl/coquimbo/ofertas_de_empleo/soldador_sin_herramientas_74023675.htm?ca=5_s&oa=74023675&xsp=41
datetime:
2020-08-28
Soldador sin herramientas
precio:15000
Ofertas de empleo
2015-09-01
https://www.yapo.cl/region_metropolitana/autos/mitsubishi_l200_katana_74023674.htm?ca=15_s&oa=74023674&xsp=42
datetime:
2020-08-28
Mitsubishi L200 katana
Autos, camionetas y 4x4
2020-03-01
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/region_metropolitana/negocios_maquinaria_construccion/congelador_4_puertas_73570964.htm?ca=15_s&oa

...

datetime:
2020-08-28
Parachoque chery tiggo 5
precio:55000
Accesorios y piezas para vehículos
2014-10-01
1062 (23000): Duplicate entry 'edward quezada _XVI_Chillán' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/consolas_videojuegos/ps4_slim_1t___2_mandos_74023936.htm?ca=15_s&oa=74023936&xsp=20
datetime:
2020-08-28
Ps4 Slim 1t + 2 mandos
precio:380000
Consolas, videojuegos y accesorios
2015-04-01
https://www.yapo.cl/region_metropolitana/instrumentos_musicales/horquilla_ciclocross_74023935.htm?ca=15_s&oa=74023935&xsp=21
datetime:
2020-08-28
Horquilla ciclocross
precio:50000
Instrumentos musicales y accesorios
2015-01-01
https://www.yapo.cl/region_metropolitana/computadores/impresora_multifuncional_hp_psc_1200_all_in_one_74023934.htm?ca=15_s&oa=74023934&xsp=22
datetime:
2020-08-28
Impresora multifuncional HP psc 1200 all-in-one
precio:14990
Computadores y accesorios
2018-11-01
https://www.yapo.cl/arica_parinacota/electrodomesticos/ventiladores_74023933.htm?ca=1_s&oa=

datetime:
2020-08-28
DEPARTAMENTO Barrio Tranquilo, Vista Cordil…
precio:360000
Arriendo - Departamento
2020-04-01
1062 (23000): Duplicate entry 'Pamela Cisternas Lillo_Región_Santiago' for key 'vendedor.PRIMARY'
https://www.yapo.cl/aisen/comprar/parcela_sector_el_salto__villa_medina__a_so__8230__73441091.htm?ca=13_s&oa=73441091&xsp=49
datetime:
2020-08-28
PARCELA Sector El Salto, Villa Medina. A so…
precio:25000000
Vendo - Terreno
2020-04-01
1062 (23000): Duplicate entry 'Pamela Cisternas Lillo_XI_Coyhaique' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/arrendar/hermosa_casa_en_valle_norte_a_70291131.htm?ca=15_s&oa=70291131&xsp=0
datetime:
2020-08-28
Hermosa Casa en Valle Norte a
precio:350000
Arriendo - Casa
2014-06-01
https://www.yapo.cl/region_metropolitana/arrendar/depto_2d_y_1b_metro_las_rejas_69897809.htm?ca=15_s&oa=69897809&xsp=1
datetime:
2020-08-28
Depto 2D y 1B Metro Las Rejas
precio:260000
Arriendo - Departamento
2014-06-01
https://www.yapo.cl/valparai

https://www.yapo.cl/region_metropolitana/muebles/comedor_madera_alerce_oregon_73990362.htm?ca=15_s&oa=73990362&xsp=33
datetime:
2020-08-28
Comedor madera alerce oregón
precio:390000
Muebles
2020-07-01
https://www.yapo.cl/biobio/negocios_maquinaria_construccion/maquina_cafe_vending_74024073.htm?ca=9_s&oa=74024073&xsp=34
datetime:
2020-08-28
Maquina Cafe Vending
precio:350000
Negocios, maquinaria y construcción
2015-03-01
https://www.yapo.cl/region_metropolitana/servicios/arriendo_de_maquinaria_pesada_73971622.htm?ca=15_s&oa=73971622&xsp=35
datetime:
2020-08-28
arriendo de maquinaria pesada
Servicios
2020-08-01
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/arica_parinacota/consolas_videojuegos/compro_psp__73905981.htm?ca=1_s&oa=73905981&xsp=36
datetime:
2020-08-28
Compro PSP
Consolas, videojuegos y accesorios
https://www.yapo.cl/region_metropolitana/animales_accesorios/busco_perro_labrador_macho_73905594.htm?ca=15_s&oa=73905594&xsp=3

datetime:
2020-08-28
CASA 18 de setiembre Peñaflor
precio:158000000
Vendo - Casa
2020-08-01
https://www.yapo.cl/region_metropolitana/electrodomesticos/teclado_marca_epic_74024059.htm?ca=15_s&oa=74024059&xsp=15
datetime:
2020-08-28
Teclado marca epic
precio:40000
Electrodomésticos
2020-08-01
https://www.yapo.cl/region_metropolitana/jardin_herramientas/macetas_de_hormigon_74024058.htm?ca=15_s&oa=74024058&xsp=16
datetime:
2020-08-28
Macetas de hormigon
precio:22000
Jardín, herramientas y exteriores
2015-07-01
1062 (23000): Duplicate entry 'saludarte_Región_Peñaflor' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/comprar/sitio_via_aurora___lo_curro_vitacura_70426302.htm?ca=15_s&oa=70426302&xsp=17
datetime:
2020-08-28
SITIO Vía Aurora - Lo Curro Vitacura
precio:25000
Vendo - Terreno
2020-08-01
https://www.yapo.cl/region_metropolitana/comprar/casa_carlos_silva_vildosola___valenzuela_ll__8230__70405471.htm?ca=15_s&oa=70405471&xsp=18
datetime:
2020-08-28
Casa carlos silva 

KeyboardInterrupt: 